In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_rows = 10

In [3]:
pd.options.display.max_rows

10

### TASK 1

In [4]:
# PARSE DATES INTO PANDAS READIBLE FORMAT
mydateparser = lambda x: pd.datetime.strptime(x, "%d.%m.%y %H:%M") if (type(x) == str and len(x) < 15) else (pd.datetime.strptime(x.split("+")[0], "%Y-%m-%dT%H:%M:%S.%f") if type(x) == str else np.nan)

In [5]:
# SELECT COLUMNS WITH DATES
test_date_columns = ['created_at', 'updated_at', 'authorized_at', 'last_event_time']
test_level_date_columns = ["created_at", "updated_at"]
class_date_columns = ["created_at", "updated_at", "latest_test_time"]

In [6]:
# READ CSVs, INSTANTLY PARSING DATES
test = pd.read_csv("test.csv", sep=";", parse_dates=test_date_columns, date_parser=mydateparser)
test_level = pd.read_csv("test_level.csv", sep=";", parse_dates=test_level_date_columns, date_parser=mydateparser)
class_ = pd.read_csv("class.csv", sep=";", parse_dates=class_date_columns, date_parser=mydateparser)

### TASK 2

not sure, which rows are "not correct". Need more info about data in here. I parsed dates into pandas date format and checked unique values of every column and to be honest did not find any "not correct" data.

### TASK 3

In [7]:
# PREPARE DFs TO MERGE
class_df = class_[["id", "name", "teaching_hours"]]
class_df = class_df.rename(columns={"id": "class_id", "name": "class_name"})

In [8]:
test_df = test[["id", "class_id", "created_at", "authorized_at", "test_level_id"]]
test_df = test_df.rename(columns={"id": "test_id", "created_at": "test_created_at", "authorized_at": "test_authorized_at", "test_level_id": "test_level"})

In [9]:
test_level_df = test_level[["id", "displayName"]]

In [10]:
test_df = pd.merge(test_df, test_level_df, left_on="test_level", right_on="id").drop(["test_level", "id"], axis=1)

In [11]:
test_df = test_df.rename(columns={"displayName": "test_level"})

In [12]:
test_utilization = pd.merge(class_df, test_df, left_on="class_id", right_on="class_id")

In [13]:
test_utilization.shape

(13039, 7)

In [14]:
# DROP NON AUTHORIZED
test_utilization = test_utilization.dropna(subset=["test_authorized_at"])

In [15]:
test_utilization.shape

(11420, 7)

In [16]:
# ENUMERATE EACH SOLVED TEST
test_utilization["class_test_number"] = test_utilization.groupby(['class_id']).cumcount().add(1)

In [17]:
test_utilization

,class_id,class_name,teaching_hours,test_id,test_created_at,test_authorized_at,test_level,class_test_number
0,1,QA Sanity,6-10,1,2018-07-06 11:37:00,2018-07-06 11:44:00,Level 1,1
1,1,QA Sanity,6-10,2,2018-07-06 11:49:00,2018-07-06 11:52:00,Level 1,2
5,1,QA Sanity,6-10,7,2018-07-06 12:17:00,2018-07-06 12:20:00,Level 1,3
14,5,new tell app release,3-5,14,2018-07-13 09:46:00,2018-07-13 09:49:00,Level 1,1
15,5,new tell app release,3-5,21,2018-07-13 15:53:00,2018-07-13 15:58:00,Level 1,2
...,...,...,...,...,...,...,...,...
13029,650,7-E,3-5,13583,2019-04-23 11:31:00,2019-04-24 08:20:00,Level 4,13
13030,650,7-E,3-5,13584,2019-04-23 11:31:00,2019-04-24 08:20:00,Level 4,14
13031,650,7-E,3-5,13585,2019-04-23 11:31:00,2019-04-24 08:20:00,Level 4,15
13035,652,ID-ALI-STEP UP,1-3,13613,2019-04-25 10:08:00,2019-04-25 10:14:00,Level 5,1


In [18]:
# CHANGE DATE FORMAT
test_utilization["test_created_at"] = test_utilization["test_created_at"].dt.strftime("%Y-%m-%d")
test_utilization["test_authorized_at"] = test_utilization["test_authorized_at"].dt.strftime("%Y-%m-%d")

In [19]:
test_utilization

,class_id,class_name,teaching_hours,test_id,test_created_at,test_authorized_at,test_level,class_test_number
0,1,QA Sanity,6-10,1,2018-07-06,2018-07-06,Level 1,1
1,1,QA Sanity,6-10,2,2018-07-06,2018-07-06,Level 1,2
5,1,QA Sanity,6-10,7,2018-07-06,2018-07-06,Level 1,3
14,5,new tell app release,3-5,14,2018-07-13,2018-07-13,Level 1,1
15,5,new tell app release,3-5,21,2018-07-13,2018-07-13,Level 1,2
...,...,...,...,...,...,...,...,...
13029,650,7-E,3-5,13583,2019-04-23,2019-04-24,Level 4,13
13030,650,7-E,3-5,13584,2019-04-23,2019-04-24,Level 4,14
13031,650,7-E,3-5,13585,2019-04-23,2019-04-24,Level 4,15
13035,652,ID-ALI-STEP UP,1-3,13613,2019-04-25,2019-04-25,Level 5,1


### TASK 4

In [20]:
test_average_scores = test_utilization.copy()

In [21]:
test_scores = test[["id", "overall_score", "test_status"]]
test_scores = test_scores.rename(columns={"id": "test_id"})

# KEEP JUST SCORING SCORED
test_scores = test_scores[test_scores["test_status"] == "SCORING_SCORED"]
test_scores = test_scores.drop("test_status", axis=1)

In [22]:
test_average_scores = test_average_scores.drop(["test_level", "class_test_number"], axis=1)

In [23]:
test_average_scores = pd.merge(test_average_scores, test_scores, on="test_id")
test_average_scores = test_average_scores.drop("test_id", axis=1)

In [24]:
test_average_scores["avg_class_test_overall_score"] = test_average_scores.groupby("class_id").mean()["overall_score"]

In [25]:
aggregation_mapping = {
    "class_name": "first",
    "teaching_hours": "first",
    "test_created_at": "first",
    "test_authorized_at": "first",
    "overall_score": "mean"
}

In [26]:
test_average_scores = test_average_scores.groupby("class_id").agg(aggregation_mapping)
test_average_scores = test_average_scores.rename(columns={"overall_score": "avg_class_test_overall_score"})

In [27]:
test_average_scores

,class_name,teaching_hours,test_created_at,test_authorized_at,avg_class_test_overall_score
class_id,,,,,
1,QA Sanity,6-10,2018-07-06,2018-07-06,16.500000
5,new tell app release,3-5,2018-07-13,2018-07-13,17.000000
6,0.6.0.0,1-3,2018-07-18,2018-07-18,21.000000
8,DCtest,11-15,2018-08-30,2019-01-29,24.000000
10,0.7.0.0,3-5,2018-08-02,2018-08-02,11.000000
...,...,...,...,...,...
647,7-B,3-5,2019-04-23,2019-04-24,27.533333
648,7-C,3-5,2019-04-23,2019-04-24,29.533333
649,7-D,3-5,2019-04-23,2019-04-24,29.545455


### TASK 5

In [28]:
# SAVE TO CSV
test_utilization.to_csv("test_utilization.csv", index=False)
test_average_scores.to_csv("test_average_scores.csv", index=False)

### TASK 6

In [ ]:
import pymysql

In [ ]:
# Connect to SQL server

In [ ]:
connection = pymysql.connect(
            host="",
            user="",
            passwd="",
            db="",
        )

In [ ]:
# Create table

with connection.cursor() as cursor:
    sql_query = ("""
            CREATE TABLE TestUtilization (
            ClassID int,
            ClassName varchar(255),
            TeachingHours varchar(10),
            TestID int,
            TestCreatedAt DATE,
            TestAuthorizedAt DATE,
            TestLevel varchar(10),
            ClassTestNumber FLOAT(5),
        );
    """
    cursor.execute(sql_query)
    self.connection.commit()

In [ ]:
df = pd.read_csv('test_utilization.csv')

In [ ]:
# Insert into table

with connection.cursor() as cursor:
    for i in range(len(df)):
        class_id = df["class_id"][i]
        class_name = df["class_name"][i]
        teaching_hours = df["teaching_hours"][i]
        test_id = df["test_id"][i]
        test_created_at = df["test_create_at"][i]
        test_authorized_at = df["test_authorized_at"][i]
        test_level = df["test_level"][i]
        class_test_number = df["class_test_number"][i]
        sql_query = (
        f"""INSERT IGNORE INTO `TestUtilization` (ClassId, ClassName, TeachingHours, TestID, 
            TestCreatedAt, TestAuthorizedAt, TestLevel, ClassTestNumber) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)""")
        val = (class_id, class_name, teaching_hours, test_id, test_created_at, test_authorized_at, test_level, class_test_number)
        cursor.execute(sql_query, val)
        connection.commit()